In [242]:
import gcp
import gcp._context
import gcp._util

import sys
import grpc.framework.face.exceptions
from grpc.beta import implementations
from gcp.spark.bytestream_pb2 import beta_create_ByteStream_stub
from gcp.spark.bytestream_pb2 import ReadRequest
from gcp.spark.bytestream_pb2 import ReadResponse
from gcp.spark.bytestream_pb2 import WriteRequest
from gcp.spark.bytestream_pb2 import WriteResponse
from gcp.spark.bytestream_pb2 import QueryWriteStatusRequest
from gcp.spark.bytestream_pb2 import QueryWriteStatusResponse
from oauth2client.client import GoogleCredentials
from grpc.beta.implementations import ClientCredentials

class MetadataTransformer(object):
    """Callable class to transform metadata for gRPC requests.
    :type client: :class:`.client.Client`
    :param client: The client that owns the cluster. Provides authorization and
                   user agent.
    """

    def __init__(self, client):
        self._credentials = client.credentials
        self._user_agent = 'GoogleCloudDataLab/1.0'

    def __call__(self, ignored_val):
        """Adds authorization header to request metadata."""
        access_token = self._credentials.get_access_token().access_token
        return [
            ('Authorization', 'Bearer ' + access_token),
            ('User-agent', self._user_agent),
        ]

SSL_CERT_FILE = '/etc/ssl/certs/ca-certificates.crt'
def get_certs():
    """Gets the root certificates.
    .. note::
        This is only called by :func:`make_stub`. For most applications,
        a few gRPC stubs (four total, one for each service) will be created
        when a :class:`.Client` is created. This function will not likely
        be used again while that application is running.
        However, it may be worthwhile to cache the output of this function.
    :rtype: str
    :returns: The root certificates for the current machine.
    """
    with open(SSL_CERT_FILE, mode='rb') as file_obj:
        return file_obj.read()
      

DEFAULT_HOST = "test-dataproc.sandbox.googleapis.com";
DEFAULT_PORT = 443;

JOB_ID = 'spark-shell-a822b38e-0a00-4949-ae5d-b76648fcadaa'
CLUSTER_ID = '39d18ff1-db0d-4305-bfb1-66dfd69e5758'

def read_bytestream(read_limit, read_offset):
  driverInputResourceUri = u'//test-dataproc.sandbox.googleapis.com/projects/datalab-spark/clusters/%s/jobs/%s/bytestreams/stdin' % (CLUSTER_ID, JOB_ID)
  driverOutputResourceUri = u'//test-dataproc.sandbox.googleapis.com/projects/datalab-spark/clusters/%s/jobs/%s/bytestreams/stdout' % (CLUSTER_ID, JOB_ID)

  custom_metadata_transformer = MetadataTransformer(gcp.Context.default())

  root_certificates = get_certs()
  client_credentials = implementations.ssl_client_credentials(root_certificates, private_key=None, certificate_chain=None)

  try:
    channel = implementations.secure_channel(DEFAULT_HOST, DEFAULT_PORT, client_credentials)
    stub = beta_create_ByteStream_stub(channel, metadata_transformer=custom_metadata_transformer)
    readRequest = ReadRequest(resource_name=driverOutputResourceUri, read_limit=read_limit, read_offset=read_offset)
    return stub.Read(readRequest, 10)
    
  except grpc.framework.interfaces.face.face.NetworkError as ex:
#  except Exception as ex:
    print('network error: ex.code:%s, ex.details:%s' % (ex.code, ex.details)) # TODO: refresh token when expired
    #gcp.Context.default().credentials.refresh(None)
    #gcp.Context._global_context = None
  return None

def query_write_status():
  driverInputResourceUri = u'//test-dataproc.sandbox.googleapis.com/projects/datalab-spark/clusters/%s/jobs/%s/bytestreams/stdin' % (CLUSTER_ID, JOB_ID)
  custom_metadata_transformer = MetadataTransformer(gcp.Context.default())

  root_certificates = get_certs()
  client_credentials = implementations.ssl_client_credentials(root_certificates, private_key=None, certificate_chain=None)
  channel = implementations.secure_channel(DEFAULT_HOST, DEFAULT_PORT, client_credentials)
  query_write_status_request = QueryWriteStatusRequest(resource_name=driverInputResourceUri)
  
  stub = beta_create_ByteStream_stub(channel, metadata_transformer=custom_metadata_transformer)
  return stub.QueryWriteStatus(query_write_status_request, 20)
    

def write_bytestream(string_data, write_offset):
  driverInputResourceUri = u'//test-dataproc.sandbox.googleapis.com/projects/datalab-spark/clusters/%s/jobs/%s/bytestreams/stdin' % (CLUSTER_ID, JOB_ID)
  driverOutputResourceUri = u'//test-dataproc.sandbox.googleapis.com/projects/datalab-spark/clusters/%s/jobs/%s/bytestreams/stdout' % (CLUSTER_ID, JOB_ID)

  custom_metadata_transformer = MetadataTransformer(gcp.Context.default())

  root_certificates = get_certs()
  client_credentials = implementations.ssl_client_credentials(root_certificates, private_key=None, certificate_chain=None)

  try:
    channel = implementations.secure_channel(DEFAULT_HOST, DEFAULT_PORT, client_credentials)
    stub = beta_create_ByteStream_stub(channel, metadata_transformer=custom_metadata_transformer)
    writeRequest = WriteRequest(resource_name=driverInputResourceUri, data = str.encode(string_data), write_offset = write_offset, finish_write = False)
   
    return stub.Write([writeRequest], 20)
    
  except grpc.framework.interfaces.face.face.NetworkError as ex:
    print('expiration error: ex.code:%s, ex.details:%s' % (ex.code, ex.details)) # TODO: refresh token when expired
    #gcp.Context.default().credentials.refresh(None)
    #gcp.Context._global_context = None
  return None


#gcp.Context.default().credentials.refresh(None)
#gcp.Context._global_context = None

#print(query_write_status())

#str.encode("data")
#print(str(query_write_status().committed_size))
#print(write_bytestream("sqlContext\n", query_write_status().committed_size))


print(ReadResponse.SerializeToString(read_bytestream(100, 0).result()))
#print(read_bytestream(5600, 10).result().data)



Rd15/12/16 04:29:06 INFO org.spark-project.jetty.server.Server: jetty-8.y.z-SNAPSHOT
15/12/16 04:29:06


In [136]:
DEFAULT_HOST = "test-dataproc.sandbox.googleapis.com";
DEFAULT_PORT = 443;




NetworkError: 

In [11]:
dir(beta_create_ByteStream_stub)

['__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__hash__',
 '__init__',
 '__module__',
 '__name__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'func_closure',
 'func_code',
 'func_defaults',
 'func_dict',
 'func_doc',
 'func_globals',
 'func_name']